### 1. Scrape Player Rankings

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np

In [2]:
site = 'https://www.espn.com/fantasy/football/story/_/page/ffcheatsheet22/fantasy-football-cheat-sheets-updated-2022-player-rankings-ppr-non-ppr-depth-charts-dynasty'
pdf = 'https://g.espncdn.com/s/ffldraftkit/22/NFLDK2022_CS_NonPPR300.pdf'

In [3]:
raw_df = pd.read_csv('top_300_2022.csv')
# raw_df = pd.read_csv('top_300_ppr_2022.csv')
raw_df = raw_df.drop([1068,1165]).reset_index(drop=True).copy()

In [4]:
rank = []
player = []
position = []
team = []
bye = []
posrank = []

for i in range(1200):
    
    if i%4 == 0:
        # rank
        rank.append(int(raw_df.iloc[i].values[0].split('.')[0]))
        position.append(raw_df.iloc[i].values[0].split('.')[1].split('(')[1].split(")")[0][0])
        tempval = raw_df.iloc[i].values[0].split('.')[1].split('(')[1].split(")")[0]
        if tempval[0] in ['Q','R','W','T']:
            if len(tempval) == 5:
                posrank.append(int(tempval[-3:]))
            if len(tempval) == 4:
                posrank.append(int(tempval[-2:]))
            if len(tempval) == 3:
                posrank.append(int(tempval[-1:]))
        if tempval[0] == 'D':
            if len(tempval) == 5:
                posrank.append(int(tempval[-2:]))
            if len(tempval) == 4:
                posrank.append(int(tempval[-1:]))
        if tempval[0] == 'K':
            if len(tempval) == 3:
                posrank.append(int(tempval[-2:]))
            if len(tempval) == 2:
                posrank.append(int(tempval[-1:]))

    if i%4 == 1:
        # name
        player.append(raw_df.iloc[i].values[0].split(', ')[0])
        team.append(raw_df.iloc[i].values[0].split(', ')[1])
    if i%4 == 2:
        # money
        pass
    if i%4 == 3:
        # bye
        bye.append(int(raw_df.iloc[i].values[0]))
            
player_rankings = pd.DataFrame({'rank':rank,'player':player,'position':position,'team':team,'bye':bye,'posrank':posrank}).sort_values(by='rank')

In [5]:
player_rankings[0:10]

,rank,player,position,team,bye,posrank
0,1,Jonathan Taylor,R,IND,14,1
4,2,Derrick Henry,R,TEN,6,2
8,3,Cooper Kupp,W,LAR,7,1
12,4,Christian McCaffrey,R,CAR,13,3
16,5,Justin Jefferson,W,MIN,7,2
20,6,Ja'Marr Chase,W,CIN,10,3
24,7,Austin Ekeler,R,LAC,8,4
28,8,Najee Harris,R,PIT,9,5
32,9,Dalvin Cook,R,MIN,7,6
36,10,Joe Mixon,R,CIN,10,7


In [6]:
player_rankings.shape

(300, 6)

### 2. Scrape Projections

#### QB Projections

In [7]:
url = 'https://www.fantasypros.com/nfl/projections/qb.php?week=draft'
response = requests.get(url)

In [8]:
soup = BeautifulSoup(response.text, "html.parser")

In [9]:
player_stat = []
attempts = []
completions = []
yards = []
touchdowns = []
interceptions = []
rush_attempts = []
rush_yards = []
rush_tds = []
fumbles = []
for i in range(30):
    name = len(str(soup.findAll('td')[4+11*i].findAll('a')[0]).split('>')[1])
    player_stat.append(str(soup.findAll('td')[4+11*i].findAll('a')[0]).split('>')[1][0:name-3])
    attempts.append(float(str(soup.findAll('td')[5+11*i]).split('>')[1].split('<')[0]))
    completions.append(float((str(soup.findAll('td')[6+11*i]).split('>')[1].split('<')[0])))
    yards.append(float(str(soup.findAll('td')[7+11*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[7+11*i]).split('>')[1].split('<')[0].split(',')[1]))
    touchdowns.append(float((str(soup.findAll('td')[8+11*i]).split('>')[1].split('<')[0])))
    interceptions.append(float((str(soup.findAll('td')[9+11*i]).split('>')[1].split('<')[0])))
    rush_attempts.append(float((str(soup.findAll('td')[10+11*i]).split('>')[1].split('<')[0])))
    rush_yards.append(float((str(soup.findAll('td')[11+11*i]).split('>')[1].split('<')[0])))
    rush_tds.append(float((str(soup.findAll('td')[12+11*i]).split('>')[1].split('<')[0])))
    fumbles.append(float((str(soup.findAll('td')[13+11*i]).split('>')[1].split('<')[0])))
    
qb_projections = pd.DataFrame({'player_stat':player_stat,'attempts':attempts,'completions':completions,'yards':yards,
                         'touchdowns':touchdowns,'interceptions':interceptions,'rush_attempts':rush_attempts,
                         'rush_yards':rush_yards,'rush_tds':rush_tds,'fumbles':fumbles})

In [10]:
qb_projections[0:10]

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Josh Allen,607.6,390.2,4406.3,34.6,13.9,116.9,662.3,6.9,3.5
1,Patrick Mahomes II,634.2,417.7,4842.1,35.9,11.6,66.5,359.7,2.8,2.8
2,Justin Herbert,625.3,415.3,4681.6,33.9,13.3,64.1,297.1,3.2,1.4
3,Lamar Jackson,520.2,336.5,3731.3,24.5,13.2,158.6,956.6,4.9,3.5
4,Kyler Murray,570.9,384.2,4258.9,27.3,12.3,107.3,551.9,5.5,2.3
5,Jalen Hurts,495.3,307.3,3645.3,24.4,12.6,144.7,772.9,7.9,2.9
6,Tom Brady,652.3,433.7,4842.8,36.3,11.9,27.9,45.7,1.7,2.4
7,Dak Prescott,593.8,399.5,4558.9,32.7,11.7,55.8,230.7,2.9,4.1
8,Joe Burrow,570.0,386.6,4574.0,31.3,12.8,46.6,187.4,2.2,1.8
9,Aaron Rodgers,559.6,377.5,4331.8,32.6,7.1,38.7,136.2,1.8,1.6


Standardize names

In [11]:
qb_projections

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Josh Allen,607.6,390.2,4406.3,34.6,13.9,116.9,662.3,6.9,3.5
1,Patrick Mahomes II,634.2,417.7,4842.1,35.9,11.6,66.5,359.7,2.8,2.8
2,Justin Herbert,625.3,415.3,4681.6,33.9,13.3,64.1,297.1,3.2,1.4
3,Lamar Jackson,520.2,336.5,3731.3,24.5,13.2,158.6,956.6,4.9,3.5
4,Kyler Murray,570.9,384.2,4258.9,27.3,12.3,107.3,551.9,5.5,2.3
5,Jalen Hurts,495.3,307.3,3645.3,24.4,12.6,144.7,772.9,7.9,2.9
6,Tom Brady,652.3,433.7,4842.8,36.3,11.9,27.9,45.7,1.7,2.4
7,Dak Prescott,593.8,399.5,4558.9,32.7,11.7,55.8,230.7,2.9,4.1
8,Joe Burrow,570.0,386.6,4574.0,31.3,12.8,46.6,187.4,2.2,1.8
9,Aaron Rodgers,559.6,377.5,4331.8,32.6,7.1,38.7,136.2,1.8,1.6


In [12]:
qb_projections['player_stat'].replace('Patrick Mahomes II','Patrick Mahomes',inplace=True)
# deshaun watson missing from projections

In [13]:
player_rankings[(player_rankings['position']=='Q')&(player_rankings['posrank']==28)]

,rank,player,position,team,bye,posrank
107,267,Deshaun Watson,Q,CLE,9,28


In [14]:
player_rankings.merge(qb_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,43,Josh Allen,Q,BUF,7,1,Josh Allen,607.6,390.2,4406.3,34.6,13.9,116.9,662.3,6.9,3.5
1,53,Patrick Mahomes,Q,KC,8,2,Patrick Mahomes,634.2,417.7,4842.1,35.9,11.6,66.5,359.7,2.8,2.8
2,54,Justin Herbert,Q,LAC,8,3,Justin Herbert,625.3,415.3,4681.6,33.9,13.3,64.1,297.1,3.2,1.4
3,62,Lamar Jackson,Q,BAL,10,4,Lamar Jackson,520.2,336.5,3731.3,24.5,13.2,158.6,956.6,4.9,3.5
4,63,Kyler Murray,Q,ARI,13,5,Kyler Murray,570.9,384.2,4258.9,27.3,12.3,107.3,551.9,5.5,2.3
5,76,Jalen Hurts,Q,PHI,7,6,Jalen Hurts,495.3,307.3,3645.3,24.4,12.6,144.7,772.9,7.9,2.9
6,89,Dak Prescott,Q,DAL,9,7,Dak Prescott,593.8,399.5,4558.9,32.7,11.7,55.8,230.7,2.9,4.1
7,90,Tom Brady,Q,TB,11,8,Tom Brady,652.3,433.7,4842.8,36.3,11.9,27.9,45.7,1.7,2.4
8,91,Aaron Rodgers,Q,GB,14,9,Aaron Rodgers,559.6,377.5,4331.8,32.6,7.1,38.7,136.2,1.8,1.6
9,92,Russell Wilson,Q,DEN,9,10,Russell Wilson,540.1,356.0,4108.4,29.0,9.6,62.7,278.9,2.1,2.3


Add scoring

In [15]:
passing_yards_score = 0.04
passing_TD_score = 4
passing_interception_score = -2
rushing_yards_score = 0.1
rushing_TD_score = 6
offense_fumble_score = -2
receptions_score = 0
receiving_yards_score = 0.1
receiving_TD_score = 6
sack_score = 1
defense_interception_score = 2
defense_fumble_score = 2
defense_TD_score = 6
safety_score = 2
points_against_score = -2/7
fg_made_score = 4
fg_miss_score = -1
xp_made_score = 1
xp_miss_score = -1

In [16]:
qb_projections['points'] = qb_projections['yards'] * passing_yards_score + qb_projections['touchdowns'] * passing_TD_score +\
    qb_projections['interceptions'] * passing_interception_score + qb_projections['rush_yards'] * rushing_yards_score +\
    qb_projections['rush_tds'] * rushing_TD_score + qb_projections['fumbles'] * offense_fumble_score

In [17]:
qb_projections.head(1)

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Josh Allen,607.6,390.2,4406.3,34.6,13.9,116.9,662.3,6.9,3.5,387.482


#### RB Projections

In [18]:
url = 'https://www.fantasypros.com/nfl/projections/rb.php?week=draft'
response = requests.get(url)

In [19]:
soup = BeautifulSoup(response.text, "html.parser")

In [20]:
player_stat = []
rush_attempts = []
rush_yards = []
rush_tds = []
receptions = []
receive_yards = []
receive_tds = []
fumbles = []
for i in range(100):
    name = len(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1])
    player_stat.append(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1][0:name-3])
    rush_attempts.append(float((str(soup.findAll('td')[5+9*i]).split('>')[1].split('<')[0])))
    if len(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]) < 6:
        rush_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0])))
    else:    
        rush_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]).split(',')[0]+(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]).split(',')[1]))
    rush_tds.append(float((str(soup.findAll('td')[7+9*i]).split('>')[1].split('<')[0])))
    receptions.append(float((str(soup.findAll('td')[8+9*i]).split('>')[1].split('<')[0])))
    receive_yards.append(float((str(soup.findAll('td')[9+9*i]).split('>')[1].split('<')[0])))
    receive_tds.append(float((str(soup.findAll('td')[10+9*i]).split('>')[1].split('<')[0])))
    fumbles.append(float((str(soup.findAll('td')[11+9*i]).split('>')[1].split('<')[0])))
    
rb_projections = pd.DataFrame({'player_stat':player_stat,'rush_attempts':rush_attempts,'rush_yards':rush_yards,
                               'rush_tds':rush_tds,'receptions':receptions,'receive_yards':receive_yards,
                               'receive_tds':receive_tds,'fumbles':fumbles})

Standardize names

In [21]:
rb_projections.iloc[50:100,]

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
50,Nyheim Hines,69.7,276.4,1.8,44.1,345.8,1.8,1.6
51,Isaiah Spiller,91.6,388.2,2.9,22.7,183.5,1.3,1.2
52,Rachaad White,93.6,374.3,3.4,21.2,168.3,1.1,1.0
53,Gus Edwards,115.3,501.7,3.5,8.9,74.9,0.2,0.7
54,Rex Burkhead,74.0,316.3,2.4,29.6,239.4,1.2,0.5
55,J.D. McKissic,51.8,215.9,1.4,44.8,348.7,1.9,0.5
56,Zamir White,92.5,393.6,3.5,13.4,110.6,0.5,1.2
57,Khalil Herbert,114.2,423.2,2.7,16.0,120.5,0.5,0.9
58,Darrel Williams,76.2,294.4,2.7,20.5,172.1,0.8,0.8
59,D'Onta Foreman,85.8,404.7,2.6,7.7,70.4,0.2,0.9


In [22]:
# deejay dalls
# dwayne washington
# rico dowdie
# kene nwangwu
# snoop conner
# qb_projections['player_stat'].replace('Patrick Mahomes II','Patrick Mahomes',inplace=True)

In [23]:
player_rankings[(player_rankings['position']=='R')&(player_rankings['posrank']==90)]

,rank,player,position,team,bye,posrank
171,283,Larry Rountree III,R,LAC,8,90


In [24]:
player_rankings.merge(rb_projections,left_on='player',right_on='player_stat').iloc[80:90,]

,rank,player,position,team,bye,posrank,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
80,265,Justin Jackson,R,DET,6,82,Justin Jackson,25.1,146.9,1.1,9.3,72.9,0.4,0.1
81,276,Jeff Wilson Jr.,R,SF,9,83,Jeff Wilson Jr.,46.4,168.5,1.5,10.1,74.2,0.6,0.6
82,280,Benny Snell Jr.,R,PIT,9,87,Benny Snell Jr.,37.6,149.0,0.9,4.4,30.3,0.1,0.2


In [25]:
rb_projections['points'] = rb_projections['rush_yards'] * rushing_yards_score + rb_projections['rush_tds'] * rushing_TD_score +\
    rb_projections['receptions'] * receptions_score + rb_projections['receive_yards'] * receiving_yards_score +\
    rb_projections['receive_tds'] * receiving_TD_score + rb_projections['fumbles'] * offense_fumble_score

In [26]:
rb_projections.head(1)

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles,points
0,Jonathan Taylor,325.4,1600.2,14.8,42.5,354.3,2.0,2.3,291.65


### WR Projections

In [27]:
url = 'https://www.fantasypros.com/nfl/projections/wr.php?week=draft'
response = requests.get(url)

In [28]:
soup = BeautifulSoup(response.text, "html.parser")

In [29]:
player_stat = []
receptions = []
receive_yards = []
receive_tds = []
rush_attempts = []
rush_yards = []
rush_tds = []
fumbles = []
for i in range(100):
    name = len(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1])
    player_stat.append(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1][0:name-3])
    receptions.append(float((str(soup.findAll('td')[5+9*i]).split('>')[1].split('<')[0])))
    if len(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]) < 6:
        receive_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0])))
    else:
        receive_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0].split(',')[1])))
    receive_tds.append(float((str(soup.findAll('td')[7+9*i]).split('>')[1].split('<')[0])))
    rush_attempts.append(float((str(soup.findAll('td')[8+9*i]).split('>')[1].split('<')[0])))
    rush_yards.append(float((str(soup.findAll('td')[9+9*i]).split('>')[1].split('<')[0])))
    rush_tds.append(float((str(soup.findAll('td')[10+9*i]).split('>')[1].split('<')[0])))
    fumbles.append(float((str(soup.findAll('td')[11+9*i]).split('>')[1].split('<')[0])))
    
wr_projections = pd.DataFrame({'player_stat':player_stat,'receptions':receptions,'receive_yards':receive_yards,
                               'receive_tds':receive_tds,'rush_attempts':rush_attempts,'rush_yards':rush_yards,
                               'rush_tds':rush_tds,'fumbles':fumbles})

Standardize names

In [30]:
wr_projections[60:120]

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
60,Corey Davis,48.0,675.2,4.0,0.0,0.0,0.0,0.5
61,Jahan Dotson,51.4,680.6,3.7,0.4,2.2,0.0,0.8
62,George Pickens,54.4,679.8,3.7,0.4,1.8,0.0,0.8
63,Van Jefferson,45.4,623.2,4.0,1.9,15.0,0.0,0.1
64,Jalen Tolbert,48.2,634.4,4.3,0.4,2.4,0.0,0.8
65,Nico Collins,51.8,679.4,3.4,0.0,0.0,0.0,0.5
66,Donovan Peoples-Jones,44.0,657.8,3.5,0.0,0.0,0.0,0.5
67,K.J. Osborn,45.5,578.4,4.6,0.2,2.0,0.0,0.4
68,Julio Jones,46.8,628.2,3.7,0.0,0.0,0.0,0.4
69,Skyy Moore,44.4,596.9,4.2,1.2,7.8,0.0,0.8


In [31]:
wr_projections['player_stat'].replace('Joshua Palmer','Josh Palmer',inplace=True)
wr_projections['player_stat'].replace('Cedrick Wilson Jr.','Cedrick Wilson',inplace=True)
# velus jones jr.
# obj
# d'wayne eskridge
# terrace marshall jr.
# bryan edwards

In [32]:
player_rankings[(player_rankings['position']=='W')&(player_rankings['posrank']==97)]

,rank,player,position,team,bye,posrank
123,271,Terrace Marshall Jr.,W,CAR,13,97


In [33]:
player_rankings.merge(wr_projections,left_on='player',right_on='player_stat')[80:100]

,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
80,226,Nick Westbrook-Ikhine,W,TEN,6,82,Nick Westbrook-Ikhine,41.2,527.1,3.8,0.0,0.0,0.0,0.4
81,227,K.J. Osborn,W,MIN,7,83,K.J. Osborn,45.5,578.4,4.6,0.2,2.0,0.0,0.4
82,229,Sterling Shepard,W,NYG,9,85,Sterling Shepard,57.7,595.5,3.1,1.0,4.2,0.0,0.5
83,230,Randall Cobb,W,GB,14,86,Randall Cobb,41.7,533.3,4.1,0.8,3.1,0.0,0.5
84,231,Corey Davis,W,NYJ,10,87,Corey Davis,48.0,675.2,4.0,0.0,0.0,0.0,0.5
85,232,A.J. Green,W,ARI,13,88,A.J. Green,36.4,524.2,2.6,0.0,0.0,0.0,0.1
86,233,Chris Conley,W,HOU,6,89,Chris Conley,33.6,411.3,2.1,0.0,0.0,0.0,0.2
87,234,Parris Campbell,W,IND,14,90,Parris Campbell,48.5,591.1,3.7,0.5,3.3,0.0,0.1
88,252,Devin Duvernay,W,BAL,10,92,Devin Duvernay,40.5,460.5,2.6,4.9,33.9,0.1,0.2
89,260,Donovan Peoples-Jones,W,CLE,9,93,Donovan Peoples-Jones,44.0,657.8,3.5,0.0,0.0,0.0,0.5


In [34]:
wr_projections['points'] = wr_projections['rush_yards'] * rushing_yards_score + wr_projections['rush_tds'] * rushing_TD_score +\
    wr_projections['receptions'] * receptions_score + wr_projections['receive_yards'] * receiving_yards_score +\
    wr_projections['receive_tds'] * receiving_TD_score + wr_projections['fumbles'] * offense_fumble_score

In [35]:
wr_projections.head(1)

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Cooper Kupp,123.0,1599.7,12.4,4.1,23.9,0.1,0.6,236.16


### TE Projections

In [36]:
url = 'https://www.fantasypros.com/nfl/projections/te.php?week=draft'
response = requests.get(url)

In [37]:
soup = BeautifulSoup(response.text, "html.parser")

In [38]:
player_stat = []
receptions = []
receive_yards = []
receive_tds = []
rush_attempts = []
rush_yards = []
rush_tds = []
fumbles = []
for i in range(80):
    name = len(str(soup.findAll('td')[3+6*i].findAll('a')[0]).split('>')[1])
    player_stat.append(str(soup.findAll('td')[3+6*i].findAll('a')[0]).split('>')[1][0:name-3])
    receptions.append(float((str(soup.findAll('td')[4+6*i]).split('>')[1].split('<')[0])))
    if len(str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0]) < 6:
        receive_yards.append(float((str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0])))
    else:
        receive_yards.append(float(str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0].split(',')[1]))
    receive_tds.append(float((str(soup.findAll('td')[6+6*i]).split('>')[1].split('<')[0])))
    fumbles.append(float((str(soup.findAll('td')[7+6*i]).split('>')[1].split('<')[0])))
    
te_projections = pd.DataFrame({'player_stat':player_stat,'receptions':receptions,'receive_yards':receive_yards,
                               'receive_tds':receive_tds,'fumbles':fumbles})

Standardize names

In [39]:
te_projections

,player_stat,receptions,receive_yards,receive_tds,fumbles
0,Travis Kelce,99.4,1179.8,9.3,0.5
1,Mark Andrews,95.5,1187.6,8.9,0.5
2,Kyle Pitts,73.1,997.9,4.7,0.5
3,Darren Waller,77.5,931.4,5.1,0.5
4,George Kittle,71.2,871.1,5.4,0.5
...,...,...,...,...,...
75,Blake Bell,12.2,112.1,0.9,0.0
76,Brock Wright,10.8,104.2,1.0,0.0
77,Zach Gentry,12.1,112.2,0.9,0.0
78,Noah Gray,10.6,100.0,1.0,0.0


In [40]:
player_rankings.merge(te_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,fumbles
0,19,Mark Andrews,T,BAL,10,1,Mark Andrews,95.5,1187.6,8.9,0.5
1,21,Travis Kelce,T,KC,8,2,Travis Kelce,99.4,1179.8,9.3,0.5
2,38,Kyle Pitts,T,ATL,14,3,Kyle Pitts,73.1,997.9,4.7,0.5
3,60,Darren Waller,T,LV,6,4,Darren Waller,77.5,931.4,5.1,0.5
4,61,George Kittle,T,SF,9,5,George Kittle,71.2,871.1,5.4,0.5
5,77,Dalton Schultz,T,DAL,9,6,Dalton Schultz,77.4,805.7,6.1,0.4
6,78,T.J. Hockenson,T,DET,6,7,T.J. Hockenson,67.6,693.2,5.1,0.4
7,87,Dallas Goedert,T,PHI,7,8,Dallas Goedert,63.5,794.5,5.0,0.5
8,88,Zach Ertz,T,ARI,13,9,Zach Ertz,64.7,663.1,4.8,0.4
9,109,Hunter Henry,T,NE,10,10,Hunter Henry,53.8,618.6,6.0,0.1


In [41]:
te_projections['points'] = te_projections['receptions'] * receptions_score +\
    te_projections['receive_yards'] * receiving_yards_score + te_projections['receive_tds'] * receiving_TD_score +\
    te_projections['fumbles'] * offense_fumble_score

In [42]:
te_projections.head(1)

,player_stat,receptions,receive_yards,receive_tds,fumbles,points
0,Travis Kelce,99.4,1179.8,9.3,0.5,172.78


### D/ST Projections

In [43]:
url = 'https://www.fantasypros.com/nfl/projections/dst.php?week=draft'
response = requests.get(url)

In [44]:
soup = BeautifulSoup(response.text, "html.parser")

In [45]:
player_stat = []
sack = []
interception = []
fumble_recovered = []
touchdowns = []
safety = []
points_against = []
yards_against = []



for i in range(32):
    name = len(str(soup.findAll('td')[0+10*i].findAll('a')[0]).split('>')[1])
    player_stat.append(str(soup.findAll('td')[0+10*i].findAll('a')[0]).split('>')[1][0:name-3])
    sack.append(float((str(soup.findAll('td')[1+10*i]).split('>')[1].split('<')[0])))
    interception.append(float((str(soup.findAll('td')[2+10*i]).split('>')[1].split('<')[0])))
    fumble_recovered.append(float((str(soup.findAll('td')[3+10*i]).split('>')[1].split('<')[0])))
    touchdowns.append(float((str(soup.findAll('td')[5+10*i]).split('>')[1].split('<')[0])))
    safety.append(float((str(soup.findAll('td')[6+10*i]).split('>')[1].split('<')[0])))
    points_against.append(float((str(soup.findAll('td')[7+10*i]).split('>')[1].split('<')[0])))
    yards_against.append((str(soup.findAll('td')[8+10*i]).split('>')[1].split('<')[0]))
    
dst_projections = pd.DataFrame({'player_stat':player_stat,'sack':sack,'interception':interception,
                               'fumble_recovered':fumble_recovered,'touchdowns':touchdowns,
                               'safety':safety,'points_against':points_against,'yards_against':yards_against})

In [46]:
for i in range(32):
    dst_projections['player_stat'][i] = dst_projections['player_stat'][i].split(' ')[-1] + ' D/ST'

C:\Users\Dan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
player_rankings.merge(dst_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
0,141,Bills D/ST,D,BUF,7,1,Bills D/ST,43.3,16.3,10.0,2.8,0.6,331.6,"5,283.0"
1,142,Saints D/ST,D,NO,14,2,Saints D/ST,44.1,15.1,9.8,2.6,0.2,339.3,"5,618.8"
2,143,Packers D/ST,D,GB,14,3,Packers D/ST,43.1,16.4,9.4,2.8,0.2,356.6,"5,594.3"
3,144,Ravens D/ST,D,BAL,10,4,Ravens D/ST,38.5,13.6,9.6,2.6,0.6,356.6,"5,782.2"
4,145,Buccaneers D/ST,D,TB,11,5,Buccaneers D/ST,44.4,14.4,11.8,2.9,0.6,366.8,"5,559.5"
5,146,49ers D/ST,D,SF,9,6,49ers D/ST,45.0,13.6,10.5,2.5,0.2,358.8,"5,516.2"
6,147,Cowboys D/ST,D,DAL,9,7,Cowboys D/ST,39.9,16.5,10.4,3.0,0.3,373.5,"5,839.8"
7,148,Colts D/ST,D,IND,14,8,Colts D/ST,36.0,15.2,11.3,2.7,0.9,364.1,"5,721.0"
8,149,Steelers D/ST,D,PIT,9,9,Steelers D/ST,48.9,14.3,10.3,2.5,0.2,392.2,"5,867.8"
9,150,Commanders D/ST,D,WAS,14,10,Commanders D/ST,39.8,13.2,10.0,2.5,0.2,389.0,"5,876.9"


In [48]:
dst_projections['points'] = dst_projections['sack'] * sack_score + dst_projections['interception'] * defense_interception_score +\
    dst_projections['fumble_recovered'] * defense_fumble_score + dst_projections['touchdowns'] * defense_TD_score +\
    dst_projections['safety'] * safety_score + dst_projections['points_against'] * points_against_score + 7

In [49]:
dst_projections.head(1)

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against,points
0,Buccaneers D/ST,44.4,14.4,11.8,2.9,0.6,366.8,"5,559.5",17.6


### K Projections

In [50]:
url = 'https://www.fantasypros.com/nfl/projections/k.php?week=draft'
response = requests.get(url)

In [51]:
soup = BeautifulSoup(response.text, "html.parser")

In [52]:
player_stat = []
fgm = []
fga = []
xp = []



for i in range(32):
    name = len(str(soup.findAll('td')[0+5*i].findAll('a')[0]).split('>')[1])
    player_stat.append(str(soup.findAll('td')[0+5*i].findAll('a')[0]).split('>')[1][0:name-3])
    fgm.append(float((str(soup.findAll('td')[1+5*i]).split('>')[1].split('<')[0])))
    fga.append(float((str(soup.findAll('td')[2+5*i]).split('>')[1].split('<')[0])))
    xp.append(float((str(soup.findAll('td')[3+5*i]).split('>')[1].split('<')[0])))
    
k_projections = pd.DataFrame({'player_stat':player_stat,'fgm':fgm,'fga':fga,'xp':xp})

In [53]:
k_projections[0:10]

,player_stat,fgm,fga,xp
0,Justin Tucker,29.9,34.2,39.9
1,Harrison Butker,27.7,31.6,46.2
2,Daniel Carlson,30.6,35.8,37.3
3,Matt Gay,29.0,34.4,41.1
4,Ryan Succop,26.6,32.6,47.6
5,Tyler Bass,27.0,33.3,45.6
6,Greg Joseph,29.8,34.9,36.9
7,Mason Crosby,26.5,32.7,45.3
8,Matt Prater,27.0,33.8,43.5
9,Evan McPherson,28.7,34.5,38.2


In [54]:
k_projections

,player_stat,fgm,fga,xp
0,Justin Tucker,29.9,34.2,39.9
1,Harrison Butker,27.7,31.6,46.2
2,Daniel Carlson,30.6,35.8,37.3
3,Matt Gay,29.0,34.4,41.1
4,Ryan Succop,26.6,32.6,47.6
5,Tyler Bass,27.0,33.3,45.6
6,Greg Joseph,29.8,34.9,36.9
7,Mason Crosby,26.5,32.7,45.3
8,Matt Prater,27.0,33.8,43.5
9,Evan McPherson,28.7,34.5,38.2


In [55]:
player_rankings.merge(k_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,fgm,fga,xp
0,151,Justin Tucker,K,BAL,10,1,Justin Tucker,29.9,34.2,39.9
1,152,Evan McPherson,K,CIN,10,2,Evan McPherson,28.7,34.5,38.2
2,153,Matt Gay,K,LAR,7,3,Matt Gay,29.0,34.4,41.1
3,154,Harrison Butker,K,KC,8,4,Harrison Butker,27.7,31.6,46.2
4,155,Daniel Carlson,K,LV,6,5,Daniel Carlson,30.6,35.8,37.3
5,156,Brandon McManus,K,DEN,9,6,Brandon McManus,27.1,32.7,40.2
6,157,Nick Folk,K,NE,10,7,Nick Folk,28.4,33.3,38.7
7,158,Tyler Bass,K,BUF,7,8,Tyler Bass,27.0,33.3,45.6
8,159,Matt Prater,K,ARI,13,9,Matt Prater,27.0,33.8,43.5
9,160,Graham Gano,K,NYG,9,10,Graham Gano,27.4,32.5,30.5


In [56]:
k_projections['points'] = k_projections['fgm'] * fg_made_score + (k_projections['fga'] - k_projections['fgm']) * fg_miss_score +\
    k_projections['xp'] * xp_made_score + k_projections['xp'] * .05 * xp_miss_score

In [57]:
k_projections.head(1)

,player_stat,fgm,fga,xp,points
0,Justin Tucker,29.9,34.2,39.9,153.205


### Combine projections

In [58]:
all_projections = pd.concat([qb_projections[['player_stat','points']],rb_projections[['player_stat','points']],wr_projections[['player_stat','points']],
          te_projections[['player_stat','points']],dst_projections[['player_stat','points']],k_projections[['player_stat','points']]])

In [59]:
full_df = player_rankings.merge(all_projections,left_on='player',right_on='player_stat')
full_df.sort_values(by='points',ascending=False,inplace=True)

In [60]:
full_df.head()

,rank,player,position,team,bye,posrank,player_stat,points
42,43,Josh Allen,Q,BUF,7,1,Josh Allen,387.482
52,53,Patrick Mahomes,Q,KC,8,2,Patrick Mahomes,361.254
53,54,Justin Herbert,Q,LAC,8,3,Justin Herbert,342.374
61,62,Lamar Jackson,Q,BAL,10,4,Lamar Jackson,338.912
62,63,Kyler Murray,Q,ARI,13,5,Kyler Murray,338.546


### Develop Model

Add/drop functions

In [61]:
def addplayer(my_team, current_df, name):
    nextplayer = current_df[current_df['player']==name]
    my_team = pd.concat([my_team,nextplayer])
    return my_team

In [62]:
def dropplayer(df, name):
    df = df[df['player']!=name]
    return df

In [63]:
def findplayer(my_team,df,position):
    test = df[(df['position']==position)]
    bye_list = list(my_team[(my_team['position']==position)]['bye'])
    for i in range(len(bye_list)):
        test = test[test['bye']!=bye_list[i]]
    return test[0:5]

Initialize draft df and my team

In [64]:
draft_df = full_df.copy()

In [65]:
my_team = full_df.copy()
my_team = my_team[my_team['player']=='a']
my_team

,rank,player,position,team,bye,posrank,player_stat,points


Initialize simulation parameters & simulate draft

In [66]:
my_team

,rank,player,position,team,bye,posrank,player_stat,points


update mypicks, fill in current rounds, roundnum, runpoints, numplayers, byelists,

below change 160 to something variable
reset bottom values to new starting values

In [67]:
def simulate(my_team,draft_df,simulations):
    current_df = draft_df.copy()
    allpicks = [1,20,21,40,41,60,61,80,81,100,101,120,121,140,141,160]
    roundnum = len(my_team)
    mypicks = (np.array(allpicks)-roundnum*10)[roundnum:17]
    picks = pd.DataFrame(columns=['Rnd1','Rnd2','Rnd3','Rnd4','Rnd5','Rnd6','Rnd7','Rnd8','Rnd9','Rnd10','Rnd11','Rnd12','Rnd13','Rnd14','Rnd15','Rnd16','Pts'],index=range(simulations))
    for i in range(len(my_team)):
        picks.iloc[:,i] = my_team['position'][i]
    currpoints = 0
    runpoints = sum(my_team['points'])
    qb_dict = dict(zip(range(16),[.93,.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    rb_dict = dict(zip(range(16),[.80,.80,.60,.30,0,0,0,0,0,0,0,0,0,0,0,0]))
    wr_dict = dict(zip(range(16),[.80,.80,.80,0.80,.30,0,0,0,0,0,0,0,0,0,0,0]))
    te_dict = dict(zip(range(16),[.80,.20,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    dst_dict = dict(zip(range(16),[.93,.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    k_dict = dict(zip(range(16),[.93,.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    num_qb = sum(my_team['position']=='Q')
    num_rb = sum(my_team['position']=='R')
    num_wr = sum(my_team['position']=='W')
    num_te = sum(my_team['position']=='T')
    num_dst = sum(my_team['position']=='D')
    num_k = sum(my_team['position']=='K')
    qb_bye = list(my_team[my_team['position']=='Q']['bye'])
    rb_bye = list(my_team[my_team['position']=='R']['bye'])
    wr_bye = list(my_team[my_team['position']=='W']['bye'])
    te_bye = list(my_team[my_team['position']=='T']['bye'])
    dst_bye = list(my_team[my_team['position']=='D']['bye'])
    k_bye = list(my_team[my_team['position']=='K']['bye'])
    position_index = []

    start = timeit.default_timer()
    for i in range(len(picks)):
        for j in range((16-roundnum)*10):
            if j not in mypicks:
                bestnum = sorted(current_df['rank'].values)[random.randint(0,2)]
                bestname = current_df[current_df['rank'] == bestnum]['player'].values[0]
                current_df = dropplayer(current_df,bestname)
            else:
                if num_qb == 0:
                    position_index.append('Q')
                if ((num_qb == 1) and (roundnum >=9)):
                    position_index.append('Q')
                if num_rb < 3:
                    position_index.append('R')
                if ((num_rb == 3) and (roundnum >=9)):
                    position_index.append('R')
                if num_wr < 4:
                    position_index.append('W')
                if ((num_wr == 4) and (roundnum >=9)):
                    position_index.append('W')
                if num_te == 0:
                    position_index.append('T')
                if ((num_te == 1) and (roundnum >=9)):
                    position_index.append('T')
                if num_dst == 0:
                    position_index.append('D')
                if ((num_dst == 1) and (roundnum >=13)):
                    position_index.append('D')
                if num_k == 0:
                    position_index.append('K')
                if ((num_k == 1) and (roundnum >=13)):
                    position_index.append('K')
                randposition = position_index[random.randint(0,len(position_index)-1)]
                if randposition == 'Q':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='Q'].iloc[k,4] in qb_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='Q'].iloc[k,1]
                            position = 'Q'
                            currpoints = current_df[current_df['position']=='Q'].iloc[k,7] * qb_dict[num_qb]
                            num_qb += 1
                            qb_bye.append(current_df[current_df['position']=='Q'].iloc[k,4])
                            break
                elif randposition == 'R':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='R'].iloc[k,4] in rb_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='R'].iloc[k,1]
                            position = 'R'
                            currpoints = current_df[current_df['position']=='R'].iloc[k,7] * rb_dict[num_rb]
                            num_rb += 1
                            rb_bye.append(current_df[current_df['position']=='R'].iloc[k,4])
                            break
                elif randposition == 'W':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='W'].iloc[k,4] in wr_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='W'].iloc[k,1]
                            position = 'W'
                            currpoints = current_df[current_df['position']=='W'].iloc[k,7] * wr_dict[num_wr]
                            num_wr += 1
                            wr_bye.append(current_df[current_df['position']=='W'].iloc[k,4])
                            break
                elif randposition == 'T':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='T'].iloc[k,4] in te_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='T'].iloc[k,1]
                            position = 'T'
                            currpoints = current_df[current_df['position']=='T'].iloc[k,7] * te_dict[num_te]
                            num_te += 1
                            te_bye.append(current_df[current_df['position']=='T'].iloc[k,4])
                            break
                elif randposition == 'D':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='D'].iloc[k,4] in dst_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='D'].iloc[k,1]
                            position = 'DST'
                            currpoints = current_df[current_df['position']=='D'].iloc[k,7] * dst_dict[num_dst]
                            num_dst += 1
                            dst_bye.append(current_df[current_df['position']=='D'].iloc[k,4])
                            break
                else:
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='K'].iloc[k,4] in k_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='K'].iloc[k,1]
                            position = 'K'
                            currpoints = current_df[current_df['position']=='K'].iloc[k,7] * k_dict[num_k]
                            num_k += 1
                            k_bye.append(current_df[current_df['position']=='K'].iloc[k,4])
                            break
                runpoints += currpoints
                picks.iloc[i,roundnum] = position
                picks.iloc[i,16] = runpoints
                roundnum += 1
                current_df = dropplayer(current_df,nextplayer)
                position_index = []
        roundnum = len(my_team)
        currpoints = 0
        runpoints = sum(my_team['points'])
        num_qb = sum(my_team['position']=='Q')
        num_rb = sum(my_team['position']=='R')
        num_wr = sum(my_team['position']=='W')
        num_te = sum(my_team['position']=='T')
        num_dst = sum(my_team['position']=='D')
        num_k = sum(my_team['position']=='K')
        qb_bye = list(my_team[my_team['position']=='Q']['bye'])
        rb_bye = list(my_team[my_team['position']=='R']['bye'])
        wr_bye = list(my_team[my_team['position']=='W']['bye'])
        te_bye = list(my_team[my_team['position']=='T']['bye'])
        dst_bye = list(my_team[my_team['position']=='D']['bye'])
        k_bye = list(my_team[my_team['position']=='K']['bye'])
        current_df = draft_df.copy()
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    picks['Pts'] = picks['Pts'].astype(float)
    return(picks.groupby(picks.columns[len(my_team)]).agg({'Pts':['count','min','mean','max']}).round())

In [68]:
# full_df.to_csv('draft_ppr.csv',index=False)

## Draft admin

Drop, simulate, find, draft, add, drop

In [69]:
# for i in range(18):
#     print(len(draft_df))
#     draft_df = dropplayer(draft_df,draft_df[draft_df['rank']==draft_df['rank'].min()]['player'].values[0])
#     print(len(draft_df))

In [ ]:
print(len(draft_df))
draft_df = dropplayer(draft_df,"")
print(len(draft_df))

In [70]:
output = simulate(my_team,draft_df,simulations=150)
output.columns = ['count','min','mean','max']
output.sort_values(by='mean',ascending=False)

Time:  42.731548100000005


,count,min,mean,max
Rnd1,,,,
R,29,1469.0,1559.0,1650.0
W,24,1450.0,1529.0,1600.0
T,21,1420.0,1506.0,1576.0
Q,25,1368.0,1498.0,1583.0
K,25,1408.0,1478.0,1540.0
DST,26,1379.0,1459.0,1545.0


In [71]:
findplayer(my_team,draft_df,'R')

,rank,player,position,team,bye,posrank,player_stat,points
0,1,Jonathan Taylor,R,IND,14,1,Jonathan Taylor,291.65
1,2,Derrick Henry,R,TEN,6,2,Derrick Henry,265.02
6,7,Austin Ekeler,R,LAC,8,4,Austin Ekeler,250.68
3,4,Christian McCaffrey,R,CAR,13,3,Christian McCaffrey,248.93
9,10,Joe Mixon,R,CIN,10,7,Joe Mixon,225.81


In [72]:
player = 'Jonathan Taylor'
my_team = addplayer(my_team,draft_df,player)
my_team.index = list(range(len(my_team)))

In [73]:
my_team

,rank,player,position,team,bye,posrank,player_stat,points
0,1,Jonathan Taylor,R,IND,14,1,Jonathan Taylor,291.65


In [74]:
print(len(draft_df))
draft_df = dropplayer(draft_df,player)
print(len(draft_df))

283
282
